In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing

# XS Generation for a 3D PWR Assembly
1. Homogenized few group parameters will be generated for a standard 3D PWR assembly
2. The assembly will experience no burnup
3. The assembly will experience no perturbations
4. The assembly will have predefined values for the axial fuel temperatures and coolant densities
taken from a thermo-hydraulic solver
5. The assembly has reflector on the top and bottom axial layers
6. The XS generation will be done using a 2 group structure where the XS will be generated for the fuel and reflector universes

## Assembly Geometry
1. The Assembly will consist of a 19 x 19 sqaure lattice
2. Each channel will have 22 axial layers, with the 1st and 22nd layer being refectors.
3. Each axial layer will have its own material and conditions pre determined by a thermal hyrdaulic solver.


In [2]:
nAxLayers = 22 #number of axial layers

#Pre generated fuel temps and coolant densities
fuelTemps = [
    669.78
    ,725.41
    ,787.58
    ,838.7
    ,878.48
    ,907.2
    ,925.58
    ,935.24
    ,937.28
    ,932.89
    ,922.53
    ,907.06
    ,887.28
    ,863.37
    ,835.66
    ,803.81
    ,769.06
    ,731.00
    ,690.12
    ,657.53
]

coolDens = [
    -0.7542
    ,-0.75224
    ,-0.74954
    ,-0.74623
    ,-0.74244
    ,-0.73828
    ,-0.73387
    ,-0.72931
    ,-0.72471
    ,-0.72013
    ,-0.71565
    ,-0.71136
    ,-0.70729
    ,-0.70353
    ,-0.70013
    ,-0.69715
    ,-0.69464
    ,-0.69265
    ,-0.69123
    ,-0.69027
]

## Generating Materials
1. Since 2 of the 22 axial layers will be reflectors, this means we will have 20 active layers. 
2. The thermohydraulic solver has pretedetermined the temperatures of the 20 active layers for the fuel and coolant densities.
3. For the fuel we will be using UO2 and lightWater for the coolant. 

In [3]:
fuelMats = [None]*len(fuelTemps)
coolMats = [None]*len(fuelTemps)

for i in range(0, len(fuelTemps)):
    fuelMats[i] = MATLIB['UO2'].duplicateMat("f"+ str(i+1))
    fuelMats[i].set('temp', fuelTemps[i])
    coolMats[i] = MATLIB['H2O'].duplicateMat("m"+ str(i+1))
    coolMats[i].set('dens', coolDens[i])
    coolMats[i].set("isBurn", False)
c1 = MATLIB['Zr'].duplicateMat("clad") # Zr cladding
c1.set("isBurn", False)
r1 = MATLIB['Reflector'].duplicateMat("reflector") # reflector plenum
r1.set("isBurn", False)
w1 = MATLIB['H2O'].duplicateMat("water")
w1.set("isBurn", False)
matList = mats()
matList.addMats(fuelMats)
matList.addMats(coolMats)
matList.addMat(c1) # Zr cladding
matList.addMat(r1) # reflector plenum
matList.addMat(w1) # water
print(matList.toString())

mat f1    -10.5216  burn 1 tmp 669.78 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f2    -10.5216  burn 1 tmp 725.41 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f3    -10.5216  burn 1 tmp 787.58 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f4    -10.5216  burn 1 tmp 838.7 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f5    -10.5216  burn 1 tmp 878.48 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f6    -10.5216  burn 1 tmp 907.2 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f7    -10.5216  burn 1 tmp 925.58 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f8    -10.5216  burn 1 tmp 935.24 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f9    -10.5216  burn 1 tmp 937.28 
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f10    -

## Generating Individual Channel Axial Layers
1. To model an individual channel axial layer we will be using the serpent pin geometry
2. Each indivial pin will have its own set of materials
3. There will be 4 types of pin models, fuel, guide, gap, and reflector pins


In [4]:
pinRadii = np.array([.4095, .4750]) #individual channel layer material region dimensions

fuelPins = []
guidePins = []
gapPins = []

for i in range(0, len(fuelMats)):
    fuelPins.append(pin("F" +str(i+1), 3))
    fuelPins[i].set('materials', np.array([fuelMats[i], c1, coolMats[i]]))
    fuelPins[i].set('radii', pinRadii)
    
    guidePins.append(pin("G" +str(i+1), 3))
    guidePins[i].set('materials', np.array([coolMats[i], c1, coolMats[i]]))
    guidePins[i].set('radii', pinRadii)
    
    gapPins.append(pin("GAP" +str(i+1), 1))
    gapPins[i].set('materials', np.array([coolMats[i]]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([r1]))

RB = pin("RB", 1) #bottom reflector
RB.set('materials', np.array([r1]))

pinList = pins()
pinList.addPins(fuelPins)
pinList.addPins(guidePins)
pinList.addPins(gapPins)
pinList.addPin(RT)
pinList.addPin(RB)

print(pinList.toString())

TypeError: materials must be of type <class 'list'> not type <class 'numpy.ndarray'>

## Generating Channels
1. To model an individual channel we will be using the serpent pinStack lattice
2. Each indivial channel will consist of indivial pins (axial layers)
3. There will be 3 types of channels fuel channels, guide tubes, and water gaps

In [ ]:
axialheights = np.array([
    -20.000
    ,0.000000
    ,18.28800
    ,36.57600
    ,54.86400
    ,73.15200
    ,91.44000
    ,109.7280
    ,128.0160
    ,146.3040
    ,164.5920
    ,182.8800
    ,201.1680
    ,219.4560
    ,237.7440
    ,256.0320
    ,274.3200
    ,292.6080
    ,310.8960
    ,329.1840
    ,347.4720
    ,365.7600
])

FCPins = fuelPins.copy()
FCPins.append(RB)
FCPins.insert(0, RT)
FCPins = np.array(FCPins)

GTPins = guidePins.copy()
GTPins.append(RB)
GTPins.insert(0, RT)
GTPins = np.array(GTPins)

WGPins = gapPins.copy()
WGPins.append(RB)
WGPins.insert(0, RT)
WGPins = np.array(WGPins)

FC = pinStack("FC", 0, 0, 22)
FC.setStack(FCPins, axialheights)

GT = pinStack("GT", 0, 0, 22)
GT.setStack(GTPins, axialheights)

WG = pinStack("WG", 0, 0, 22)
WG.setStack(WGPins, axialheights)

latList = lats()
latList.addLats([FC, GT, WG])

print(latList.toString())

print(FC._pinsDict.toString())

## Generating Assembly Layout
1. To model the fuel assembly we will be using a sqaure lattice (19 x 19).
2. The assembly will consist of our generated fuel, guide tube, and water gap channels.

In [ ]:
FA = sqLat("FA", 0, 0, 19, 1.260)

FAMap = np.array([
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
])

FA.setMap(FAMap)
print(FA.toString())

## Generating Assembly Housing
1. Housing can be either just the closing surface ("border") for the core or can include sheilding, baffle, vessel, barrel, etc.
2. The option for having concentric rings around the assembly housing is also an option.
2. For this scenario the assembly will be enclosed by concentric rings.

In [ ]:
width = 250
height  = 455
radii = [24, 25, 26, 27]
mats = [c1, w1, c1]

h1 = housing(width=width, height=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)

print(h1.toString())

## Putting it all together
1. The core/assembly layout (main lattice) must be given.
2. The channels must be given (pinStack objs)
3. The channels axial layers must be given. (pin objs)
4. The materials must be given.
5. The border surface must be given.
6. The optional flags parameter can be set `for additional settings

In [ ]:
input = core(mainUniv=FA, housing=h1, lats=latList, pins=pinList, materials=matList)

In [ ]:
inventory = np.array([541350, 922340, 922350, 942390])
burnPoints = np.array([0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 1.1E5, 1.2E5])
input.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)

print(input.burnup['toString'])


In [ ]:
ngroups = 2
ebounds = np.array([0.625E-6])
xsUnivs = [FA]

input.setXS(ngroups=ngroups, ebounds=ebounds, universes= xsUnivs, setFPPXS=True, setADF=True)
print(input.xs['toString'])


In [ ]:
input.setXSLib("", "lwj3.11t")

In [ ]:

input.writeFile("test.txt")

text_file = open("test.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()

###  Core Object Info

In [ ]:
vars(input)